<a href="https://colab.research.google.com/github/cocoisland/DS-Unit-2-Sprint-4-Model-Validation/blob/master/pump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install kaggle

In [6]:
from google.colab import drive
drive.mount('/content/drive')
%env KAGGLE_CONFIG_DIR=/content/drive/My Drive/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
env: KAGGLE_CONFIG_DIR=/content/drive/My Drive/


In [7]:
!kaggle  competitions download -c ds1-predictive-modeling-challenge

  0% 0.00/236k [00:00<?, ?B/s]
100% 236k/236k [00:00<00:00, 76.3MB/s]
  0% 0.00/948k [00:00<?, ?B/s]
100% 948k/948k [00:00<00:00, 46.0MB/s]
  0% 0.00/211k [00:00<?, ?B/s]
100% 211k/211k [00:00<00:00, 58.9MB/s]
  0% 0.00/3.81M [00:00<?, ?B/s]
100% 3.81M/3.81M [00:00<00:00, 124MB/s]


In [8]:
!unzip train_features.csv.zip
!unzip train_labels.csv.zip
!unzip test_features.csv.zip



Archive:  train_features.csv.zip
  inflating: train_features.csv      
Archive:  train_labels.csv.zip
  inflating: train_labels.csv        
Archive:  test_features.csv.zip
  inflating: test_features.csv       


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.feature_selection as fe
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif
from sklearn.feature_selection import RFECV

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_validate

from mlxtend.plotting import plot_decision_regions
#import category_encoders as ce


In [13]:
# DO NOT USE
# date as index
df_train = pd.read_csv('train_features.csv',index_col='date_recorded', 
              parse_dates=True, infer_datetime_format=True)
df_test = pd.read_csv('test_features.csv', index_col='date_recorded',
                     parse_dates=True, infer_datetime_format=True)
df_label = pd.read_csv('train_labels.csv')

df_train.shape, df_test.shape, df_label.shape


((59400, 39), (14358, 39), (59400, 2))

In [26]:
# id as index
df_train = pd.read_csv('train_features.csv',index_col='id')
df_test = pd.read_csv('test_features.csv', index_col='id')
df_label = pd.read_csv('train_labels.csv')
df_sample = pd.read_csv('sample_submission.csv')

df_train.shape, df_test.shape, df_label.shape


((59400, 39), (14358, 39), (59400, 2))

In [349]:
df_sample.head()

,id,status_group
0,50785,functional
1,51630,functional
2,17168,functional
3,45559,functional
4,49871,functional


In [27]:
X_train_val, X_test, y_train_val, y_test = train_test_split(
        df_train, df_label, test_size=0.25, random_state=42, shuffle=True)

X_train_val.shape, X_test.shape, y_train_val.shape, y_test.shape

((44550, 39), (14850, 39), (44550, 2), (14850, 2))

In [0]:
# majority baseline
# balanced data - logistic class_weight = 
#            (None, accuracy = 0.55),
#            (balanced,accuracy=0.39-0.4)


df_label['status_group'].value_counts(normalize=True)

In [0]:
pd.set_option('display.max_columns', None)

#df_str=X_train_val.select_dtypes(exclude=['int','float']).copy()
df_str=X_train_val.select_dtypes(object).copy()
df_str.head(3)


In [0]:
# As observations increase, more features will fail to pass the threshold
# At 50,000+ observation, passing threshold drops to 0.1

#remove all features that are either one or zero (on or off) in more than 80% of the samples.
#threshold = 0.8 * (1- .8)

import sklearn.feature_selection as fe

def variance_threshold_selector(data, threshold=0.01):
    selector = fe.VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

In [0]:
def wrangle(X, y):
  X_dummies = pd.get_dummies(X.funder)
  dummy_filter=variance_threshold_selector(X_dummies)
  
  X_num = X._get_numeric_data()
  
  #y = y.replace(['non functional','functional','functional needs repair'],[1,2,3])
  #y = y.values
  
  return X_num.join(dummy_filter, how='outer'), y.status_group.values

In [0]:
# NOT USE - for understanding variance_threshold only
# same as variance threshold function
def noise_filter(df, col, threshold=0.01):
  X_dummies=pd.DataFrame()
  X_dummies[col]=df[col].value_counts(normalize=True)

  for noise_cat in X_dummies.loc[(X_dummies[col] < threshold)].index.values:
    df[col].replace(noise_cat, 'noise', inplace=True)

  #df_str['funder'].replace('0', 'noise', inplace=True)
  #df_str['funder'].replace(np.nan,'noise' ,inplace=True)

  return pd.get_dummies(df[col])

dummy_filter = noise_filter(df_str, 'funder' ,0.01)
dummy_filter=dummy_filter.drop(['0','noise'], axis=1)
dummy_filter.shape

In [32]:
# Baseline metric - numeric only - test_score=0.55
X, y= wrangle(X_train_val, y_train_val)


scores = cross_validate(LogisticRegression(solver='lbfgs',class_weight=None), X, y, 
                        scoring='accuracy', cv=3,
                        return_train_score=True, return_estimator=True)

pd.DataFrame(scores)

/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/linear_model/logistic.py:758: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


,estimator,fit_time,score_time,test_score,train_score
0,"LogisticRegression(C=1.0, class_weight=None, d...",1.686140,0.034812,0.554979,0.551702
1,"LogisticRegression(C=1.0, class_weight=None, d...",2.045398,0.024234,0.554747,0.560438
2,"LogisticRegression(C=1.0, class_weight=None, d...",1.988147,0.024203,0.550744,0.552035


In [0]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, y_pred)

In [41]:
# Define an estimator and param_grid
from sklearn.feature_selection import SelectPercentile, SelectKBest, f_classif,mutual_info_classif

#X,y = wrangle(X_train_val, y_train_val)
X,y = wrangle(X_test, y_test)


pipe = make_pipeline(
    RobustScaler(), 
    #fe.SelectKBest(mutual_info_classif), #slow 0.58
    fe.SelectKBest(f_classif),
    LogisticRegression(solver='lbfgs'))

param_grid = {
    'selectkbest__k': ['all'],
    'logisticregression__class_weight': [None, 'balanced'],
    'logisticregression__C': [.01, .1, 1.0, 10.0]
}

# Fit on the train set, with grid search cross-validation
gs = GridSearchCV(pipe, param_grid=param_grid, cv=3, 
                  scoring='accuracy', return_train_score=True,
                  verbose=1)

gs.fit(X, y)
validation_score = gs.best_score_
print()
print('Cross-Validation Score:', validation_score)
print()
print('Best estimator:', gs.best_estimator_)
print()

results = pd.DataFrame(gs.cv_results_)
print('Best result from grid search of {} parameter combinations'.format(len(results)))
results.sort_values(by='rank_test_score').head(3)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:   17.1s finished


()
('Cross-Validation Score:', 0.5947474747474748)
()
('Best estimator:', Pipeline(memory=None,
     steps=[('robustscaler', RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True)), ('selectkbest', SelectKBest(k='all', score_func=<function f_classif at 0x7fee0d6a61b8>)), ('logisticregression', LogisticRegression(C=10.0, class_weight=None, dual=False, fit...enalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False))]))
()
Best result from grid search of 8 parameter combinations


,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_logisticregression__C,param_logisticregression__class_weight,param_selectkbest__k,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
6,0.693448,0.009533,0.594747,0.596465,10,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",1,0.594627,0.597232,0.584242,0.599394,0.605375,0.592768,0.009626,0.000087,0.008627,0.002759
4,0.692240,0.009448,0.591717,0.594916,1,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",2,0.593819,0.596222,0.582828,0.598687,0.598505,0.589839,0.006861,0.000053,0.006570,0.003728
2,0.692040,0.009464,0.591178,0.593536,0.1,None,all,"{u'selectkbest__k': u'all', u'logisticregressi...",3,0.589982,0.596525,0.584444,0.594343,0.599111,0.589738,0.005865,0.000092,0.006047,0.002829


In [0]:
#Which features were selected?


selector = gs.best_estimator_.named_steps['selectkbest']
all_names = X.columns.values
selected_mask = selector.get_support()
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print('\n')
print('Features not selected:')
for name in unselected_names:
    print(name)


In [0]:
X,y=wrangle(X_train_val, y_train_val)

In [0]:
pd.DataFrame(y).replace(['non functional','functional','functional needs repair'],[1,2,3])

In [0]:
from sklearn.ensemble import RandomForestRegressor as RFR

X,y=wrangle(X_train_val, y_train_val)
names = X.columns

y = pd.DataFrame(y).replace(['non functional','functional','functional needs repair'],[1,2,3])
y = y.values

# Fitting a random forest regression
rfr = RFR().fit(X, y)

# Creating scores from feature_importances_ ranking (some randomness here)
rfr_scores = pd.Series(data=rfr.feature_importances_, name='RFR', index=names)

rfr_scores

In [0]:
from sklearn.feature_selection import RFECV
from sklearn.linear_model import Lasso

X,y= wrangle(X_train_val, y_train_val)


#y = y.replace(['non functional','functional','functional needs repair'],[1,2,3])
#y = y.values

X_scaled = RobustScaler().fit_transform(X)

# Lasso can't handle multi-class label.
#rfe = RFECV(Lasso(), scoring='accuracy', cv=3)
rfe = RFECV(LogisticRegression(), scoring='accuracy', cv=3)
X_subset = rfe.fit_transform(X_scaled, y)

all_names = X.columns
selected_mask = rfe.support_
selected_names = all_names[selected_mask]
unselected_names = all_names[~selected_mask]

print('Features selected:')
for name in selected_names:
    print(name)

print()
print('Features not selected:')
for name in unselected_names:
    print(name)


In [42]:
X,y=wrangle(X_test, y_test)
X.shape, y.shape

((14850, 25), (14850,))

In [43]:
X.head(3)

,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year,0,...,Government Of Tanzania,Hesawa,Kkkt,Norad,Private Individual,Rwssp,Tasaf,Unicef,World Bank,World Vision
id,,,,,,,,,,,,,,,,,,,,,
37098,0.0,0,31.985658,-3.596360,0,17,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14530,0.0,0,32.832815,-4.944937,0,14,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
62607,10.0,1675,35.488289,-4.242048,0,21,1,148,2008,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
# Predict with X_test features
X,y=wrangle(X_test, y_test)
y_pred = gs.predict(X)

# Compare predictions to y_test labels
test_score = accuracy_score(y_test.status_group, y_pred)
print('Test Score:', test_score)

('Test Score:', 0.5958922558922559)


In [45]:
#submission as format as df_sample
y_submission = y_test.copy()
y_submission['status_group']=y_pred
y_submission.shape

(14850, 2)

In [46]:
y_submission.head(3)

,id,status_group
2980,37098,functional
5246,14530,functional
22659,62607,functional


In [0]:
y_submission=y_submission.reset_index(drop=True)

In [49]:
y_submission.head()

,id,status_group
0,37098,functional
1,14530,functional
2,62607,functional
3,46053,functional
4,47083,functional


In [0]:
from google.colab import files

y_submission.to_csv('data.csv')
files.download("data.csv")